<a href="https://colab.research.google.com/github/deviknitkkr/ML-Models/blob/main/LSTM-Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from prophet import Prophet
from datetime import datetime, timedelta


In [72]:
ticker = yf.Ticker("SBIN.NS")
time_period = '15mo'
df = ticker.history(period=time_period)

prediction_point = 80
train_size = int(len(df) * 0.85)
factor = 600
look_back = 80

In [73]:
#@title
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.add_trace(go.Scatter(x=df.index, y=df['Close'].rolling(window=1).mean(), name='SMA 1'))

fig.update_layout(title='Stock Data (Last {})'.format(time_period),
                  yaxis_title='Price ($)',
                  xaxis_rangeslider_visible=True)
pyo.iplot(fig)

In [ ]:
# LSTM model
data = df['Close'].values
train_data = [row/factor for row in data[:train_size]]

trainX, trainY = [], []
for i in range(len(train_data)-look_back):
    a = train_data[i:(i+look_back)]
    trainX.append(a)
    trainY.append(train_data[i + look_back])

trainX = np.array(trainX)
trainY = np.array(trainY)

model = Sequential()
model.add(LSTM(500, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(LSTM(500, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the LSTM model
model.fit(trainX, trainY, epochs=30, batch_size=1, verbose=1)
trainPredict = model.predict(trainX)

Epoch 1/30
 91/185 [=============>................] - ETA: 2:07 - loss: 0.0551

In [ ]:
# Prophet model

df_prophet = df.reset_index()[['Date', 'Close']].rename({'Date':'ds', 'Close':'y'}, axis=1)
df_prophet['ds'] = pd.to_datetime(df_prophet['ds']).dt.tz_localize(None) # Remove timezone
df_prophet = df_prophet.iloc[:train_size]

# Exclude Saturdays and Sundays
df_prophet = df_prophet[df_prophet['ds'].dt.weekday < 5]

p_model = Prophet(changepoint_prior_scale=0.05,
                holidays_prior_scale=5,
                seasonality_prior_scale=10,
                changepoint_range=.999,
                daily_seasonality=False,
                weekly_seasonality=True,
                yearly_seasonality=True)

p_model.add_country_holidays(country_name='IN')
p_model.fit(df_prophet)

future = p_model.make_future_dataframe(periods= prediction_point, freq='D')
future = future[future['ds'].dt.weekday < 5]

ForcastedProphet = p_model.predict(future)

In [ ]:
pred_start_date = df.index[len(train_data)-1]
future_dates = pd.date_range(start=pred_start_date, end=pred_start_date + timedelta(days=prediction_point), freq='B')
future_values = train_data.copy()

for i in range (1,len(future_dates)):
  input = np.array(future_values[-look_back-1:-1])
  input = input.reshape((1, look_back, 1))
  yhat = model.predict(input, verbose=0)
  future_values.append(yhat[0][0])


In [ ]:
fig = go.Figure()

TrainedX = [row for row in df.index[look_back:trainPredict.shape[0]+look_back]]
TrainedY = [row[0]*factor for row in trainPredict]
TrainedDF = pd.DataFrame({'Close':TrainedY}, index = TrainedX)

ForcastedX = future_dates
ForcastedY = [row*factor for row in future_values[:len(future_dates)]]
ForcastedDF = pd.DataFrame({'Close': ForcastedY}, index=ForcastedX)

fig.add_trace(go.Scatter(x=df.index, y=df['Close'].rolling(window=1).mean(), name='Actual'))
fig.add_trace(go.Scatter(x = TrainedDF.index, y = TrainedDF['Close'].rolling(window=1).mean(), name='Trained'))
fig.add_trace(go.Scatter(x=ForcastedDF.index, y = ForcastedDF['Close'].rolling(window=1).mean(), name='LSTM Forcast'))
fig.add_trace(go.Scatter(x=ForcastedProphet['ds'], y=ForcastedProphet['yhat'], name='Prophet Forcast'))
fig.add_vline(x=datetime.today(), line=dict(color='gray', dash='dot'))

fig.update_layout(title='Apple Stock Data (Last {})'.format(time_period),
                  yaxis_title='Price ($)',
                  xaxis_rangeslider_visible=True)
pyo.iplot(fig)